In [7]:
pip install azureml-sdk --upgrade

     |████████████████████████████████| 107 kB 5.0 MB/s eta 0:00:01
     |████████████████████████████████| 2.1 MB 9.6 MB/s eta 0:00:01
     |████████████████████████████████| 185 kB 30.5 MB/s eta 0:00:01
     |████████████████████████████████| 8.6 MB 22.1 MB/s eta 0:00:01
     |████████████████████████████████| 306 kB 54.6 MB/s eta 0:00:01
     |████████████████████████████████| 62 kB 372 kB/s  eta 0:00:01
ERROR: azureml-widgets 1.17.0 has requirement azureml-core~=1.17.0, but you'll have azureml-core 1.18.0 which is incompatible.
ERROR: azureml-widgets 1.17.0 has requirement azureml-telemetry~=1.17.0, but you'll have azureml-telemetry 1.18.0 which is incompatible.
ERROR: azureml-train-automl 1.17.0 has requirement azureml-automl-core~=1.17.0, but you'll have azureml-automl-core 1.18.0 which is incompatible.
ERROR: azureml-train-automl 1.17.0 has requirement azureml-dataset-runtime[fuse,pandas]~=1.17.0, but you'll have azureml-dataset-runtime 1.18.0 which is incompatible.
ERROR: azure

In [8]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-125517")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-125517
Azure region: southcentralus
Subscription id: 3ec87f2b-33cf-4aed-b36f-750175813524
Resource group: aml-quickstarts-125517


In [9]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
aml_compute_target = "cpu-cluster"
try:
  aml_compute = AmlCompute(ws, aml_compute_target)
  print("Found existing compute target!")
except ComputeTargetException:
  print("Creating new compute cluster...")
  provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2", min_nodes = 1, max_nodes = 4)
  aml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)
  aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

print("Azure Machine Learning Compute Cluster Created!")

Found existing compute target!
Azure Machine Learning Compute Cluster Created!


In [10]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        "--C": uniform(0.01, 1.0),
        "--max_iter": uniform(10, 1000)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

import shutil

# the training logic is in the train.py file, therefore, Copying the train.py in the training folder.
shutil.copy('train.py', './training')

# defining a conda environment YAML file with the training script dependencies
#%%writefile conda_dependencies.yml

#dependencies:
#  -python=3.6.2
#  -scikit-learn
#  -pip:
#    -azureml-defaults

# creating an Azure ML environment
from azureml.core import Environment
sklearn_env = Environment.get(workspace=ws, name='AzureML-Tutorial')

# Create an SKLearn estimator for use with train.py
est = SKLearn(source_directory="./training",
              compute_target=aml_compute,
              entry_script='train.py',
              environment_definition=sklearn_env)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                     hyperparameter_sampling=ps, 
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=12,
                                     max_concurrent_runs=4,
                                     policy=policy)

WARNING - If environment_definition or conda_dependencies_file_path is specified, Azure ML will not install any framework related packages on behalf of the user.


In [11]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [ ]:
hyperdrive_run.wait_for_completion(show_output=True)

In [ ]:
assert(hyperdrive_run.get_status() == "Completed")

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])

#list the model files uploaded during the run
print(best_run.get_file_names())

#register the folder as a model
model = best_run.register_model(model_name='hyperdrive-model', model_path='outputs/model.joblib')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = from_delimited_files(path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv", separator=',', encoding='utf8')

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=x,
    label_column_name=y,
    n_cross_validations=10,
    compute_target=aml_compute,
    num_classes=2,
    max_concurrent_iterations=4)

In [ ]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(config=automl_config, show_output=True)

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_automl_run = automl_run.get_best_run_by_primary_metric()
print(best_automl_run.get_details()['runDefinition']['arguments'])

#list the model files uploaded during the run
print(best_automl_run.get_file_names())

#register the folder as a model
model = best_automl_run.register_model(model_name='automl-model', model_path='outputs/automl-model.joblib')

In [ ]:
# Clean up deployed resources.

aml_compute.delete()